# Orquestrador de Chatbots - Etapa 4
-----------------------------------
## Obter os Classificadores Finais

1. Montar um **VotingClassifier** multiclasse com os classificadores -base.
2. Montar N **VotingClassifier**, um específico para cada classe (bot).
3. Comparar as métricas para todos os modelos gerados usando a base de treino e teste.
4. Persistir os classificadores obtidos.

In [1]:
%load_ext autoreload
%autoreload 2

## Bibliotecas utilizadas

In [2]:
import pandas as pd
import numpy as np
import csv
import codecs
import os
import glob
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, make_scorer

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import json

In [3]:
import orquestrador_funcoes_gerais as ofg

## Funções Específicas

In [4]:
def gerar_voting(clfs_selec, clfs_base, voting='soft'):
    estimators =[]
    for clf in clfs_base:
        if clf.__class__.__name__ in clfs_selec:
            if (voting=='soft') and (clf.__class__.__name__ in ['LinearSVC','SGDClassifier']):
                estimators.append((clf.__class__.__name__, CalibratedClassifierCV(clf, method='sigmoid', cv=3)))
            else:
                estimators.append((clf.__class__.__name__, clf))
                        
    voting_clf = VotingClassifier(estimators=estimators, n_jobs=-1, voting=voting)
    return voting_clf

## Configurações

In [5]:
cfg = ofg.carregar_configuracoes()

Conferir as configurações antes de prosseguir
nome_arquivo_configuracao: config.json
------------------------------------------------------------------------------------------------------------------------
aplicar_stemmer: False
------------------------------------------------------------------------------------------------------------------------
bots: [{'bot_id': 1, 'nome': 'Alistamento Militar', 'arquivo': 'skill-alistamento-militar.json'}, {'bot_id': 2, 'nome': 'COVID', 'arquivo': 'skill-covid.json'}, {'bot_id': 3, 'nome': 'Login Único', 'arquivo': 'skill-login-unico.json'}, {'bot_id': 4, 'nome': 'IRPF 2020', 'arquivo': 'skill-perguntao-irpf-2020.json'}, {'bot_id': 5, 'nome': 'PGMEI', 'arquivo': 'skill-pgmei.json'}, {'bot_id': 6, 'nome': 'Selo Turismo Responsável', 'arquivo': 'skill-poc-selo-turismo-responsavel.json'}, {'bot_id': 7, 'nome': 'Cadastur', 'arquivo': 'skill-cadastur.json'}, {'bot_id': 8, 'nome': 'Tuberculose', 'arquivo': 'skill-tuberculose.json'}]
---------------------

In [6]:
bots=cfg['bots']

In [7]:
print('ATENÇÃO!!! Aplicação de Stemmer =', cfg['aplicar_stemmer'])

ATENÇÃO!!! Aplicação de Stemmer = False


## Carregar dados processados

Vide etapa 2 para ver como os dados foram processados.

In [8]:
arquivo_treino_testes_processado = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_treino_testes_processado']) 
df = pd.read_csv(arquivo_treino_testes_processado, index_col=None, engine='python', sep =',', encoding="utf-8")
print('Total de registros carregados:',len(df), 'de', cfg['arquivo_treino_testes_processado'])
df.tail(-1)

Total de registros carregados: 2631 de treino_testes_processado.csv


,bot_id,pergunta
1,8,aperto mao transmite tuberculose
2,6,diaria tera valor maior aderir selo turismo re...
3,3,preciso conta acesso login unico
4,3,resolver problema cpf invalido
5,4,perda total carro declarar recebimento seguro
...,...,...
2626,4,filho dependente
2627,4,contribuinte obrigado preenchimento numero recibo
2628,5,preciso imprimir guia microempreendedor indivi...
2629,7,quer dizer cnae


## Carregar Vetorizador

Vide etapa 2 para ver como o vetorizador foi gerado.

In [9]:
arquivo_vetorizador = os.path.join(os.getcwd(), cfg['diretorio_modelos'], cfg['arquivo_vetorizador'])
print('Carregando Vetorizador --->',arquivo_vetorizador,'\n')
try:            
    file = open(arquivo_vetorizador, 'rb')
    vectorizer = pickle.load(file)
    file.close()
except Exception as e:
    print('Erro no carregamento do vetorizador',arquivo_vetorizador,'-->',str(e))
print(vectorizer)

Carregando Vetorizador ---> E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\vetorizador.pkl 

TfidfVectorizer(ngram_range=[1, 2], smooth_idf=False, sublinear_tf=True)


## Carregar Classificadores Base

Vide etapa 3 para ver como os classificadores base foram obtidos

In [10]:
print('Carregando os classificadores base disponíveis')
print('-'*120)
clfs_base = []
padrao_arquivo_classificador_base = cfg['padrao_arquivo_classificador_base'].replace('%classe%','*')
padrao_arquivo_classificador_base = os.path.join(os.getcwd(), cfg['diretorio_modelos'], padrao_arquivo_classificador_base)

# Busca todos os arquivos que atendam ao padrão
all_files = glob.glob(padrao_arquivo_classificador_base)
if all_files == []:
   print('Não foi encontrado nenhum arquivo que atenda ao padrão', padrao_arquivo_classificador_base)
else:
    for arquivo in all_files:
        try:            
            file = open(arquivo, 'rb')
            clf = pickle.load(file)
            file.close()
            clfs_base.append(clf)
            print('Carregado:',clf)
        except Exception as e:
            print('Erro no carregamento do modelo',arquivo,'-->',str(e))
        print('-'*120)

Carregando os classificadores base disponíveis
------------------------------------------------------------------------------------------------------------------------
Carregado: LinearSVC(C=5, class_weight='balanced', loss='hinge', max_iter=2000,
          random_state=112020)
------------------------------------------------------------------------------------------------------------------------
Carregado: LogisticRegression(C=5, class_weight='balanced', max_iter=1000,
                   random_state=112020, solver='saga')
------------------------------------------------------------------------------------------------------------------------
Carregado: RandomForestClassifier(criterion='entropy', max_depth=160, n_estimators=1600,
                       n_jobs=-1, random_state=112020)
------------------------------------------------------------------------------------------------------------------------
Carregado: SGDClassifier(alpha=0.0005, class_weight='balanced', loss='modified_huber

## Carrega informações de processamento de outros módulos

In [11]:
arquivo_informacoes = os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_informacoes'])
info = json.loads(open(arquivo_informacoes).read())
info

{'scores': {'LogisticRegression': 0.8805013839861138,
  'LinearSVC': 0.8877512938837833,
  'SVC': 0.8810979465853578,
  'SGDClassifier': 0.8878559780805455,
  'RandomForestClassifier': 0.8081470154783211}}

## **Voting Classifier** Multiclasse

In [12]:
bot_id_gs = 0 # Se = 0, usa todas as classes (multiclasse).
if bot_id_gs != 0:
    df['classe'] = df['bot_id'].apply(lambda x : 1 if x == bot['id'] else 0)
    y_GS = df['classe'].tolist()
else:
    y_GS = df['bot_id'].tolist()

X_GS = df['pergunta'].tolist()

In [13]:
# Parâmetros de Grid Search do VotingClassifier
param_grid = {'classifier__weights':[[2,1,1,1,2],[1,1,1,1,1],[2.5,3,1,3,3],[3,3.1,1.2,1,3.1,3.1]]}

clfs_selec = []
for clf in clfs_base:
    clfs_selec.append(clf.__class__.__name__)

In [14]:
# Grid Search
clf = gerar_voting(clfs_selec, clfs_base)
estimator, results = ofg.executa_grid_search(param_grid, clf, X_GS,  y_GS, vectorizer)
info['scores'][estimator['classifier'].__class__.__name__ + '_Multiclasse'] = results.iloc[0]['mean_test_score']
results.head(10)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:   15.5s remaining:   15.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   16.4s finished


VotingClassifier - Média Score: 0.8849724430971689 
Params: {'classifier__weights': [2.5, 3, 1, 3, 3]}


,index,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,2,14.518541,0.429997,0.720702,0.001995,"[2.5, 3, 1, 3, 3]","{'classifier__weights': [2.5, 3, 1, 3, 3]}",0.885330,0.874847,0.894740,0.884972,0.008125,1
1,0,14.398853,0.425893,0.689620,0.050686,"[2, 1, 1, 1, 2]","{'classifier__weights': [2, 1, 1, 1, 2]}",0.882004,0.872701,0.898809,0.884505,0.010804,2
2,1,14.434192,0.382509,0.722127,0.002550,"[1, 1, 1, 1, 1]","{'classifier__weights': [1, 1, 1, 1, 1]}",0.881083,0.874847,0.896122,0.884017,0.008930,3
3,3,0.036840,0.002593,0.000000,0.000000,"[3, 3.1, 1.2, 1, 3.1, 3.1]","{'classifier__weights': [3, 3.1, 1.2, 1, 3.1, ...",NaN,NaN,NaN,NaN,NaN,4


In [15]:
print('Scores Multiclasse na Base de Treino/Testes')
print('-'*50)
for score in info['scores']:
    print('%-30s' % score,info['scores'][score])

Scores Multiclasse na Base de Treino/Testes
--------------------------------------------------
LogisticRegression             0.8805013839861138
LinearSVC                      0.8877512938837833
SVC                            0.8810979465853578
SGDClassifier                  0.8878559780805455
RandomForestClassifier         0.8081470154783211
VotingClassifier_Multiclasse   0.8849724430971689


In [16]:
info['scores']

{'LogisticRegression': 0.8805013839861138,
 'LinearSVC': 0.8877512938837833,
 'SVC': 0.8810979465853578,
 'SGDClassifier': 0.8878559780805455,
 'RandomForestClassifier': 0.8081470154783211,
 'VotingClassifier_Multiclasse': 0.8849724430971689}

## Voting Classifiers por Classe (por Bot)

Cria um Voting Classifier específico para cada classe (ou seja, para cada bot)

In [17]:
# Parâmetros de Grid Search do VotingClassifier
param_grid = {'classifier__weights':[[2,1,1,1,2],[1,1,1,1,1],[2.5,3,1,3,3],[3,3.1,1.2,1,3.1,3.1],[5,3.1,1.2,1,3.1,3.1]]}

clfs_selec = []
for clf in clfs_base:
    clfs_selec.append(clf.__class__.__name__)

In [18]:
soma_ponderada = 0
X_train = df['pergunta'].tolist() 
for bot in bots:
    print('Fazendo Grid Search para Bot',bot['bot_id'],'-',bot['nome'])
    
    # Gera os rótulos 0/1 das classes, de acordo com o bot corrente
    df['classe'] = df['bot_id'].apply(lambda x : 1 if x == bot['bot_id'] else 0)  
    y_train =  df['classe'].tolist()
    
    # Faz o GridSearch
    clf = gerar_voting(clfs_selec, clfs_base)
    estimator, results = ofg.executa_grid_search(param_grid, clf, X_train,  y_train, vectorizer)
    
    bot['clf'] = estimator['classifier']
    bot['results'] = results
    
    print('Bot',bot['nome'],'\nMédia Score:',results.iloc[0]['mean_test_score'],'- Params:',results.iloc[0]['params'])
    print('-'*120,'\n\n')
    
    soma_ponderada += len(df[df['bot_id']==bot['bot_id']]) * results.iloc[0]['mean_test_score']
    
media_ponderada = soma_ponderada/len(df)   
info['scores'][estimator['classifier'].__class__.__name__ + '_por_classe'] = media_ponderada
print('Fim processamento')

Fazendo Grid Search para Bot 1 - Alistamento Militar
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:    0.2s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    5.7s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    6.2s finished


VotingClassifier - Média Score: 0.9637151853321173 
Params: {'classifier__weights': [2, 1, 1, 1, 2]}
Bot Alistamento Militar 
Média Score: 0.9637151853321173 - Params: {'classifier__weights': [2, 1, 1, 1, 2]}
------------------------------------------------------------------------------------------------------------------------ 


Fazendo Grid Search para Bot 2 - COVID
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:    0.3s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    5.7s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    6.1s finished


VotingClassifier - Média Score: 0.9328625871893461 
Params: {'classifier__weights': [2, 1, 1, 1, 2]}
Bot COVID 
Média Score: 0.9328625871893461 - Params: {'classifier__weights': [2, 1, 1, 1, 2]}
------------------------------------------------------------------------------------------------------------------------ 


Fazendo Grid Search para Bot 3 - Login Único
Fitting 3 folds for each of 5 candidates, totalling 15 fits


...

VotingClassifier - Média Score: 0.9227552448991627 
Params: {'classifier__weights': [2, 1, 1, 1, 2]}
Bot Selo Turismo Responsável 
Média Score: 0.9227552448991627 - Params: {'classifier__weights': [2, 1, 1, 1, 2]}
------------------------------------------------------------------------------------------------------------------------ 


Fazendo Grid Search para Bot 7 - Cadastur
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:    0.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    9.3s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   10.7s finished


VotingClassifier - Média Score: 0.795508208846547 
Params: {'classifier__weights': [1, 1, 1, 1, 1]}
Bot Cadastur 
Média Score: 0.795508208846547 - Params: {'classifier__weights': [1, 1, 1, 1, 1]}
------------------------------------------------------------------------------------------------------------------------ 


Fazendo Grid Search para Bot 8 - Tuberculose
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:    0.3s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    5.1s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    5.6s finished


VotingClassifier - Média Score: 0.9911434143400221 
Params: {'classifier__weights': [2, 1, 1, 1, 2]}
Bot Tuberculose 
Média Score: 0.9911434143400221 - Params: {'classifier__weights': [2, 1, 1, 1, 2]}
------------------------------------------------------------------------------------------------------------------------ 


Fim processamento


In [28]:
print('-'*36)
print('Scores Classificadores:Treino/Testes')
print('-'*36)
for score in info['scores']:
    print('%-30s' % score, round(info['scores'][score],3))
print('-'*36)

------------------------------------
Scores Classificadores:Treino/Testes
------------------------------------
LogisticRegression             0.881
LinearSVC                      0.888
SVC                            0.881
SGDClassifier                  0.888
RandomForestClassifier         0.808
VotingClassifier_Multiclasse   0.885
VotingClassifier_por_classe    0.951
------------------------------------


## Persiste os **Voting Classifier** por classe

In [20]:
# Persistindo os classificadores
print('%-30s' % 'Bot','Arquivo')
print('-'*120)
for bot in bots:
    clf = bot['clf']
    arquivo_classificador_voting = cfg['padrao_arquivo_classificador_voting'].replace('%bot_id%',str(bot['bot_id']))
    arquivo_classificador_voting = os.path.join(os.getcwd(), cfg['diretorio_modelos'], arquivo_classificador_voting)
    with open(arquivo_classificador_voting, "wb") as clf_file:
        pickle.dump(clf, clf_file)
    print('%-30s' % (str(bot['bot_id']) + ' - ' + bot['nome']), arquivo_classificador_voting)

Bot                            Arquivo
------------------------------------------------------------------------------------------------------------------------
1 - Alistamento Militar        E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_1.pkl
2 - COVID                      E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_2.pkl
3 - Login Único                E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_3.pkl
4 - IRPF 2020                  E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_4.pkl
5 - PGMEI                      E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_5.pkl
6 - Selo Turismo Responsável   E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_6.pkl
7 - Cadastur                   E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_7.pkl
8 - Tuberculose                E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\c

In [21]:
# Persiste os scores em um arquivo json
arquivo_informacoes = os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_informacoes'])
with open(arquivo_informacoes, 'w') as fp:
    json.dump(info, fp, indent=2)
print('Informações atualizadas em', arquivo_informacoes)

Informações atualizadas em E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\info.json


In [22]:
print('Fim da etapa 4!')

Fim da etapa 4!
